In [72]:
%load_ext autoreload
%autoreload 2

from FP567_Lib import *

import tensorflow_hub as hub
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import json

# improve/change plot appearance
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
'''
After having their html removed, some of the update files are useless. Lets seperate them
'''
move_file_to_if_fnx(
    PATH_TO_RAW_GAME_UPDATES,
    PATH_TO_NOISY_JUNK_UPDATES,
    check_if_init_is_game_updates)

In [ ]:
'''
resoureces/raw_updates/
    by_year_updates
    by_day_updates
    game_updates
are the same updates, just oranized differently. 
Let's use the by_year_updates file and make some specfic yy/mm/dd_update files.
The by_year_updates have a certain form/pattern.
'''
unprocd_year_updates = os.path.join(PATH_TO_RAW_UPDATES, "by_year_updates")
for file in os.listdir(unprocd_year_updates):
    print(file[:4])
    yy = file[2:4]
    lines = open(os.path.join(unprocd_year_updates, file), "r").readlines()
    i = 0
    while i < len(lines):
        toks = lines[i].split()
        if len(toks) == 2:
            maybe_month = toks[1].lower()
            if toks[0] == "##" and maybe_month in MONTHS:
                print(f"\t{maybe_month}")
                path_to_month_folder = os.path.join(
                    PATH_TO_PROCESSED_UPDATES,
                    yy,
                    word_month_to_number_month(maybe_month))
                os.makedirs(path_to_month_folder, exist_ok=True)
                i = i + 2
                update_line_toks = lines[i].split()
                while len(update_line_toks) > 0:
                    d = update_line_toks[1][:-2]
                    print(f"\t\t{d}")
                    # write this update as its own file at yy/m/d.
                    # if in here, maybe_month is now for sure a month
                    with open(os.path.join(path_to_month_folder, f"{d}.txt"), 'w') as f:
                        f.write(" ".join(update_line_toks[3:]))

                    i = i + 1
                    update_line_toks = lines[i].split()
        i = i + 1


In [ ]:
'''
Found another common pattern amongst some files with some extractable updates that are different than
those that we already got from above notebook cells.
The below code parses this pattern, saving the updates.
First, to more easily keep track of what I have procd and not, move those that I process over to
a "has_been_processed" sub folder.
'''
files_with_pattern = [
    "Combat_Achievements.html.clean",
    "Achievement_Diary.html.clean",
    "Polls.html.clean",
    "Shooting_Stars.html.clean"]
get_updates_from_certain_misc_updates_pattern(files_with_pattern)

In [ ]:
'''
Another found pattern of updates to extract:
## Year

day month
update
'''
files_with_pattern = ["Hidden_updates.html.clean"]
get_updates_from_other_certain_misc_updates_pattern(files_with_pattern)

In [29]:
'''
There is a table-like list of update title and their release dates. Let's include these titles
with the updates as they are somewhat informative.
'''
table = pd.read_table(
    os.path.join(PATH_TO_RAW_MISC_UPDATES_BEEN_PROCD, "postbag_table.txt"), sep='|')
table.columns = list(map(str.strip, table.columns))

In [ ]:
table[["Title", "Date"]].apply(save_table_as_updates, axis=1)

In [ ]:
'''
Now, lets take all the organized txt files and combine those that represent the same days,
then embedd and save these.
'''
encoder_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embedder = hub.load(encoder_url)

In [ ]:
for year_directory in os.listdir(PATH_TO_PROCESSED_UPDATES_BY_YEAR):
    current_year_path = os.path.join(PATH_TO_PROCESSED_UPDATES_BY_YEAR, year_directory)
    # print(current_year_path)
    for month_directory in os.listdir(current_year_path):
        current_month_path = os.path.join(current_year_path, month_directory)
        # print(current_month_path)
        files = os.listdir(current_month_path)
        day_paths = [os.path.join(current_month_path, dd) for dd in files]
        dirs = [d_path for d_path in day_paths if '.' not in d_path and os.path.isdir(d_path)]
        for day_path in day_paths:
            days_embedded = []
            # just to make sure these are the .txt files i want
            if day_path.endswith(".txt"):
                try:
                    text = open(day_path, 'r').read()
                except:
                    print("Error on ", day_path)
                    continue
                # check if this file also has a directory
                day_dir_path = day_path.split('.')[0]
                days_embedded.append(day_dir_path)
                # just to make sure these are the directories i want
                if day_dir_path in dirs and not day_dir_path.endswith(".embedding") and not day_dir_path.endswith(".txt"):
                    for another_of_same_day_file_txt in os.listdir(day_dir_path):
                        if another_of_same_day_file_txt.endswith(".txt"):
                            text = text + f" {open(os.path.join(day_dir_path, another_of_same_day_file_txt), 'r').read()}"
                # embed and save this text embedding
                np.savetxt(f"{day_dir_path}.embedded", embedder([text]), delimiter=',')

            # for any days saved strictly as a directory (not up-one-level day.txt + day directory)
            for day_dir_path in [d for d in dirs if d not in days_embedded]:
                text = ""
                # just to make sure these are the directories i want
                if not day_dir_path.endswith(".embedding") and not day_dir_path.endswith(".txt"):
                    for another_of_same_day_file_txt in os.listdir(day_dir_path):
                        if another_of_same_day_file_txt.endswith(".txt"):
                            text = text + f" {open(os.path.join(day_dir_path, another_of_same_day_file_txt), 'r').read()}"
                    # embed and save this text embedding
                    np.savetxt(f"{day_dir_path}.embedded", embedder([text]), delimiter=',')